Project: Image Super Resolution

In [ ]:
#this is a colab project file.
import torch
from google.colab import drive
drive.mount('/content/drive/')

# **ESRGAN:**

In [ ]:
#imports for colab env.
import cv2
import os
import time
from time import sleep
from tqdm import tqdm
!sudo apt install tesseract-ocr
!pip install pytesseract

from PIL import Image
import numpy as np
import tensorflow as tf
import pytesseract
from pytesseract import Output
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


#test image
testLrPath = "low-res image path" 

In [1]:
#imports for local env.
"""import cv2
import numpy as np
import os
from time import sleep
from tqdm import tqdm
import pytesseract
from pytesseract import Output


pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

testLrPath = "low-res image path" """

In [ ]:
#read and preprocess image for Model
lr_image = cv2.imread(testLrPath, cv2.IMREAD_COLOR)
lr_image = cv2.cvtColor(lr_image, cv2.COLOR_BGR2RGB)
if (lr_image.shape[2] == 4):
  lr_image = lr_image[:,:,:3]

lr_image = tf.cast(lr_image, tf.float32)
lr_image = tf.expand_dims(lr_image, 0)

In [ ]:
LRimg = cv2.imread(testLrPath, cv2.IMREAD_COLOR)
cv2_imshow(LRimg)
orgImageHeight = LRimg.shape[0]
orgImageWidth = LRimg.shape[1]
print("Original Image Height: ", orgImageHeight)
print("Original Image Width: ", orgImageWidth)

In [ ]:
esrganModel = hub.load("/content/drive/MyDrive/ESRGAN_Model") #load ESRGAN Model

In [ ]:
hr_image = esrganModel(lr_image)
hr_image = tf.squeeze(hr_image)

In [ ]:
# Display and Save Super Resolution Image
image = np.asarray(tf.squeeze(hr_image))
image = np.clip(image, 0, 255)
image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
plt.imshow(image)

image = tf.squeeze(hr_image)
if not isinstance(image, Image.Image):
  image = np.clip(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
savePath = "save path of super-res image"
image.save(savePath)

#Restart Runtime and Run again if error pops-up

# **Scaling with Nearest Neighbour, Liner and Cubic Interpolation:**

In [ ]:
lowResImg = cv2.imread(testLrPath, cv2.IMREAD_COLOR)

supResPath = 'super-res image path' #loading saved super-res image
supResImg = cv2.imread(supResPath, cv2.IMREAD_COLOR)

orgImageHeight = lowResImg.shape[0]
orgImageWidth = lowResImg.shape[1]
print("Original Image Height: ", orgImageHeight)
print("Original Image Width: ", orgImageWidth)

imageHeight = supResImg.shape[0]
imageWidth = supResImg.shape[1]
print("SR Image Height: ", imageHeight)
print("SR Image Width: ", imageWidth)

In [ ]:
#code for performing nearest Upscaling, linear Upscaling, cubic Upscaling.
scaleFactor = 4
new_width = orgImageWidth * scaleFactor
new_height = orgImageHeight * scaleFactor
new_dim = (new_width, new_height)

nearestUpscaled = cv2.resize(lowResImg, new_dim, interpolation = cv2.INTER_NEAREST)

linearUpscaled = cv2.resize(lowResImg, new_dim, interpolation= cv2.INTER_LINEAR)

cubicUpscaled = cv2.resize(lowResImg, new_dim, interpolation= cv2.INTER_CUBIC)

In [ ]:
#pytesseract output for Upscaling, linear Upscaling, cubic Upscaling and ESRGAN images
img1 = cv2.cvtColor(nearestUpscaled, cv2.COLOR_BGR2RGB)
outData1 = pytesseract.image_to_string(img1)
print("Nearest Neighbour Data:", outData1)

img2 = cv2.cvtColor(linearUpscaled, cv2.COLOR_BGR2RGB)
outData2 = pytesseract.image_to_string(img2)
print("Linear Upscaled Data:", outData2)

img3 = cv2.cvtColor(cubicUpscaled, cv2.COLOR_BGR2RGB)
outData3 = pytesseract.image_to_string(img3)
print("Cubic Upscaled Data:", outData3)

img4 = cv2.cvtColor(supResImg, cv2.COLOR_BGR2RGB)
outData4 = pytesseract.image_to_string(img4)
print("ESRGAN Data:", outData4)

In [ ]:
#bouding box detected text in super-res image
d = pytesseract.image_to_data(img4, output_type=Output.DICT)
n_boxes = len(d['level'])
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    cv2.rectangle(img4, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2_imshow(img4)


# **PSNR Evaluation:**

In [ ]:
evalImgPath = 'path of high-res evaluation image'
evalImg = cv2.imread(evalImgPath, cv2.IMREAD_COLOR)
print(evalImg.shape)
cv2_imshow(evalImg)

In [ ]:
#resize/convert to low res for evaluation
lrEvalSize = (320, 200)
lrEvalImg = cv2.resize(evalImg, lrEvalSize)
print(lrEvalImg.shape)
cv2_imshow(lrEvalImg)

In [ ]:
if (lrEvalImg.shape[2] == 4):
  lrEvalImg = lrEvalImg[:,:,:3]

lrEvalImg = tf.cast(lrEvalImg, tf.float32)
lrEvalImg = tf.expand_dims(lrEvalImg, 0)

if (evalImg.shape[2] == 4):
  evalImg = evalImg[:,:,:3]

evalImg = tf.cast(evalImg, tf.float32)
evalImg = tf.expand_dims(evalImg, 0)


In [ ]:
evalSupResImg = esrganModel(lrEvalImg)

In [ ]:
evalSupResImg = tf.squeeze(evalSupResImg)
image = np.asarray(tf.squeeze(evalSupResImg))
image = np.clip(image, 0, 255)
image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
EvalsavePath = "Save path for super-res evalution image"
image.save(savePath)

In [ ]:
evalOut = tf.image.psnr(np.clip(evalSupResImg, 0, 255), np.clip(evalImg, 0, 255), max_val=255)
print("PSNR Evaluation: %.3f" %evalOut)